In [9]:
import pandas as pd
from pathlib import Path
from os.path import abspath, dirname, split, splitext
data_path = abspath("data")
import os
import ipywidgets as widgets
import numpy as np
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename
import time
import sqlite3 as sql
pd.set_option('display.max_rows', 20)

# Track and Data parameters

In [26]:
column_names = ['start_date', 'start_time', 'lap_number', 'session_time', 'lap_time',
       'long_acceleration', 'lattitude', 'gps_lat_acceleration',
       'heading', 'rpm', 'gps_distance', 'gps_long_acceleration',
       'lat_acceleration', 'longitude', 'gps_speed',
       'num_satellites', 'throttle_position','file']

In [27]:
track_configs = {'umc-east':
                 [{'corner_num':'1','corner_name':'turn_1','coord':[40.58313387564972, -112.377462146332],'heading':'w'},
                  {'corner_num':'2','corner_name':'fast','coord':[40.582147300913256, -112.37800820636099],'heading':'e'},
                  {'corner_num':'3','corner_name':'faster','coord':[40.58179961496041, -112.37656498144997],'heading':'e'},
                  {'corner_num':'4','corner_name':'gotcha','coord':[40.580491356707405, -112.3748967376012],'heading':'e'},
                  {'corner_num':'5','corner_name':'maybe_yll_makit','coord':[40.57961917032409, -112.37351558687992],'heading':'e'},
                  {'corner_num':'6','corner_name':'satisfaction','coord':[40.578511241494205, -112.37304227118328],'heading':'s'},
                  {'corner_num':'7','corner_name':'agony','coord':[40.5792478983455, -112.37545540530871],'heading':'n'},
                  {'corner_num':'7','corner_name':'ecstacy','coord':[40.5805, -112.377],'heading':'w'},
                  {'corner_num':'8','corner_name':'first_attitude','coord':[40.57857072875101, -112.37753089860107],'heading':'s'},
                  {'corner_num':'9','corner_name':'second_attitude','coord':[40.5774, -112.376],'heading':'e'},
                  {'corner_num':'10','corner_name':'bad_attitude','coord':[40.57694360871717, -112.37537781257156],'heading':'s'},
                  {'corner_num':'11','corner_name':'tooele_turn','coord':[40.5766960843945, -112.37324915989871],'heading':'e'},
                  {'corner_num':'12','corner_name':'kink','coord':[40.57793369685003, -112.37137402733451],'heading':'n'},
                  {'corner_num':'13','corner_name':'club_house_corner','coord':[40.579784179430945, -112.37164560191455],'heading':'n'},
                  {'corner_num':'14','corner_name':'wind_up','coord':[40.58097459021327, -112.3728482893404],'heading':'w'},
                  {'corner_num':'15','corner_name':'release','coord':[40.5821119431023, -112.37338367922673],'heading':'e'},
                  {'corner_num':'0','corner_name':'main_straight','coord':[40.58319624080327, -112.37422168078798],'heading':'w'}
                 ],
                'umc-west':
                 [{'corner_num':'1','corner_name':'sunset_bend','coord':[40.58311944287482, -112.38493614471426],'heading':'w'},
                  {'corner_num':'2','corner_name':'dreamboat','coord':[40.581444976283656, -112.38602512153257],'heading':'s'},
                  {'corner_num':'3','corner_name':'workout','coord':[40.579399707182475, -112.38554232391608],'heading':'s'},
                  {'corner_num':'4','corner_name':'scream','coord':[40.57737188978102, -112.38559998100673],'heading':'s'},
                  {'corner_num':'5','corner_name':'black_rock_hairpin','coord':[40.57687030209047, -112.38212941425624],'heading':'e'},
                  {'corner_num':'6','corner_name':'right_hook','coord':[40.57786280175162, -112.38142687037963],'heading':'w'},
                  {'corner_num':'6','corner_name':'knockout','coord':[40.578375053880364, -112.38314107743857],'heading':'n'},
                  {'corner_num':'7','corner_name':'demon','coord':[40.579164768227564, -112.38111775107392],'heading':'e'},
                  {'corner_num':'7','corner_name':'devil','coord':[40.57992245836512, -112.37934734050484],'heading':'n'},
                  {'corner_num':'7','corner_name':'diablo','coord':[40.580605439303724, -112.37961430717796],'heading':'n'},
                  {'corner_num':'8','corner_name':'indecision','coord':[40.58069081143074, -112.38092103878846],'heading':'w'},
                  {'corner_num':'9','corner_name':'precision','coord':[40.58109632754622, -112.38319728094869],'heading':'n'},
                  {'corner_num':'10','corner_name':'west_ten','coord':[40.582366217373085, -112.38167978617521],'heading':'e'}, 
                  {'corner_num':'0','corner_name':'main_straight','coord':[40.58311453419377, -112.38143864891444],'heading':'w'}
                 ],
                'umc-outer':
                 [{'corner_num':'1','corner_name':'sunset_bend','coord':[40.58311944287482, -112.38493614471426],'heading':'w'},
                  {'corner_num':'2','corner_name':'dreamboat','coord':[40.581444976283656, -112.38602512153257],'heading':'s'},
                  {'corner_num':'3','corner_name':'workout','coord':[40.579399707182475, -112.38554232391608],'heading':'s'},
                  {'corner_num':'4','corner_name':'scream','coord':[40.57737188978102, -112.38559998100673],'heading':'s'},
                  {'corner_num':'5','corner_name':'black_rock_hairpin','coord':[40.57687030209047, -112.38212941425624],'heading':'e'},
                  {'corner_num':'6','corner_name':'right_hook','coord':[40.57786280175162, -112.38142687037963],'heading':'w'},
                  {'corner_num':'6','corner_name':'knockout','coord':[40.578375053880364, -112.38314107743857],'heading':'n'},
                  {'corner_num':'7','corner_name':'witchcraft','coord':[40.579164768227564, -112.38111775107392],'heading':'e'},
                  {'corner_num':'8','corner_name':'first_attitude','coord':[40.57857072875101, -112.37753089860107],'heading':'s'},
                  {'corner_num':'9','corner_name':'second_attitude','coord':[40.5774, -112.376],'heading':'e'},
                  {'corner_num':'10','corner_name':'bad_attitude','coord':[40.57694360871717, -112.37537781257156],'heading':'s'},
                  {'corner_num':'11','corner_name':'tooele_turn','coord':[40.5766960843945, -112.37324915989871],'heading':'e'},
                  {'corner_num':'12','corner_name':'kink','coord':[40.57793369685003, -112.37137402733451],'heading':'n'},
                  {'corner_num':'13','corner_name':'club_house_corner','coord':[40.579784179430945, -112.37164560191455],'heading':'n'},
                  {'corner_num':'14','corner_name':'wind_up','coord':[40.58097459021327, -112.3728482893404],'heading':'w'},
                  {'corner_num':'15','corner_name':'release','coord':[40.5821119431023, -112.37338367922673],'heading':'e'},
                  {'corner_num':'0','corner_name':'main_straight','coord':[40.58319624080327, -112.37422168078798],'heading':'w'}
                 ],
                'umc-full':
                 [{'corner_num':'1','corner_name':'sunset_bend','coord':[40.58311944287482, -112.38493614471426],'heading':'w'},
                  {'corner_num':'2','corner_name':'dreamboat','coord':[40.581444976283656, -112.38602512153257],'heading':'s'},
                  {'corner_num':'3','corner_name':'workout','coord':[40.579399707182475, -112.38554232391608],'heading':'s'},
                  {'corner_num':'4','corner_name':'scream','coord':[40.57737188978102, -112.38559998100673],'heading':'s'},
                  {'corner_num':'5','corner_name':'black_rock_hairpin','coord':[40.57687030209047, -112.38212941425624],'heading':'e'},
                  {'corner_num':'6','corner_name':'right_hook','coord':[40.57786280175162, -112.38142687037963],'heading':'w'},
                  {'corner_num':'6','corner_name':'knockout','coord':[40.578375053880364, -112.38314107743857],'heading':'n'},
                  {'corner_num':'7','corner_name':'demon','coord':[40.579164768227564, -112.38111775107392],'heading':'e'},
                  {'corner_num':'7','corner_name':'devil','coord':[40.57992245836512, -112.37934734050484],'heading':'n'},
                  {'corner_num':'7','corner_name':'diablo','coord':[40.580605439303724, -112.37961430717796],'heading':'n'},
                  {'corner_num':'8','corner_name':'indecision','coord':[40.58069081143074, -112.38092103878846],'heading':'w'},
                  {'corner_num':'9','corner_name':'precision','coord':[40.58109632754622, -112.38319728094869],'heading':'n'},
                  {'corner_num':'10','corner_name':'fast','coord':[40.582147300913256, -112.37800820636099],'heading':'e'},
                  {'corner_num':'11','corner_name':'faster','coord':[40.58179961496041, -112.37656498144997],'heading':'e'},
                  {'corner_num':'12','corner_name':'gotcha','coord':[40.580491356707405, -112.3748967376012],'heading':'e'},
                  {'corner_num':'13','corner_name':'maybe_yll_makit','coord':[40.57961917032409, -112.37351558687992],'heading':'e'},
                  {'corner_num':'14','corner_name':'satisfaction','coord':[40.578511241494205, -112.37304227118328],'heading':'s'},
                  {'corner_num':'15','corner_name':'agony','coord':[40.5792478983455, -112.37545540530871],'heading':'n'},
                  {'corner_num':'15','corner_name':'ecstacy','coord':[40.5805, -112.377],'heading':'w'},
                  {'corner_num':'16','corner_name':'first_attitude','coord':[40.57857072875101, -112.37753089860107],'heading':'s'},
                  {'corner_num':'17','corner_name':'second_attitude','coord':[40.5774, -112.376],'heading':'e'},
                  {'corner_num':'18','corner_name':'bad_attitude','coord':[40.57694360871717, -112.37537781257156],'heading':'s'},
                  {'corner_num':'19','corner_name':'tooele_turn','coord':[40.5766960843945, -112.37324915989871],'heading':'e'},
                  {'corner_num':'20','corner_name':'kink','coord':[40.57793369685003, -112.37137402733451],'heading':'n'},
                  {'corner_num':'21','corner_name':'club_house_corner','coord':[40.579784179430945, -112.37164560191455],'heading':'n'},
                  {'corner_num':'22','corner_name':'wind_up','coord':[40.58097459021327, -112.3728482893404],'heading':'w'},
                  {'corner_num':'23','corner_name':'release','coord':[40.5821119431023, -112.37338367922673],'heading':'e'},
                  {'corner_num':'0','corner_name':'main_straight','coord':[40.58319624080327, -112.37422168078798],'heading':'w'}
                 ]
                }

# Functions

In [30]:
speed_units = 'kph' #'kph' or 'mph'

In [1]:
def mph_convert(speed_column, units = 'kph'):
    if units == 'kph':
        speed = speed_column * 0.62137
    else:
        speed = speed_column
    return speed

In [2]:
def time_conv(time_col):
    seconds_list = []
    minutes_list = []
    for time in time_col.to_list():
        seconds = time/1000000000
        minutes = int(seconds//60)
        remaining_seconds = seconds -minutes*60
        whole_seconds = str(int(remaining_seconds)).zfill(2)
        partial_seconds = str(remaining_seconds%1)[1:5]
        minutes_seconds = "%s:%s%s" % (minutes, whole_seconds, partial_seconds)
        seconds_list.append(seconds)
        minutes_list.append(minutes_seconds)
    times = {'seconds':seconds_list,'minutes':minutes_list}
    return times


# Data grab and prep

In [24]:
def tsv_import(filename,meta_id, speed_units,column_lookup):
    df_final = pd.read_csv(filename, delimiter = "\t")
    new_cols = []
    for col_name in df_final.columns:
        new_cols.append(column_lookup[col_name.lower()])
    df_final.columns = new_cols
    df_final['upload_meta_id'] = meta_id
    df_final['mph'] = mph_convert(df_final['gps_speed'],speed_units)
    return df_final


In [4]:
def vbo_import(filename,meta_id, speed_units):    
    with open(filename) as f:
        for x, line in enumerate(f):
            #print(line)
            header = False
            comments = False
            session_data = False
            laptiming = False
            column_names = False
            data = False
            if x == 0:
                file_create = line
            elif line.strip() == '[header]':
                header, comments, session_data, laptiming, column_names, data = True, False, False, False, False, False
                header_data = []
            elif line.strip() == '[comments]':
                header, comments, session_data, laptiming, column_names, data = False, True, False, False, False, False
                comments_data = []
            elif line.strip() == '[session data]':
                header, comments, session_data, laptiming, column_names, data = False, False, True, False, False, False
                session_data = []
            elif line.strip() == '[laptiming]':
                header, comments, session_data, laptiming, column_names, data = False, False, False, True, False, False 
                laptiming_data = []
            elif line.strip() == '[column names]':
                header, comments, session_data, laptiming, column_names, data = False, False, False, False, True, False 
                column_names_data = []
                column_line = x
            elif line.strip() == '[data]':
                header, comments, session_data, laptiming, column_names, data = False, False, False, False, False, True
                lap_data = []
                data_line = x
            if header:
                header_data.append(line)
            elif comments:
                comments_data.append(line)
            elif session_data:
                session_data.append(line)
            elif laptiming:
                laptiming_data.append(line)
            elif column_names:
                if column_line != x:
                    column_names_data.append(line)
            elif data:
                if data_line != x:
                    lap_data.append(line.split('\t'))
        raw_data_dict = {}    
        raw_data_dict['file_create'] = file_create
        raw_data_dict['header'] = header_data
        raw_data_dict['comments'] = comments_data
        raw_data_dict['session_data'] = session_data
        raw_data_dict['laptiming'] = laptiming_data
        raw_data_dict['column_names'] = column_names_data
        raw_data_dict['data'] = lap_data
    return raw_data_dict


In [5]:
def csv_import(filename):
    string = "LAPINDEX	DATE	TIME	TIME_LAP	LATITUDE	LONGITUDE	SPEED_KPH	SPEED_MPH	HEIGHT_M	HEIGHT_FT	HEADING_DEG	GPSDIFFERENTIAL[UNKNOWN/2D3D/DGPS/INVALID]	GPSFIX[NOFIX/2D/3D/VIRTUAL/INDOOR]	GPSINTERPOLATED[FALSE/TRUE]	SATELLITES	HDOP	ACCURACY_M	DISTANCE_KM	DISTANCE_MILE	ACCELERATIONSOURCE[CALCULATED/MEASURED/UNDEFINED]	LATERALG	LINEALG	LEAN	RPM	MAF	WHEEL_SPEED_KPH	WHEEL_SPEED_MPH	THROTTLE	GEAR	FUEL	COOLANT_CELSIUS	OIL_CELSIUS	IAT_CELSIUS	MAP	OIPPRESSURE	BRAKEPRESSURE	WHEEL_SPEED_RL_DELTA_KPH	WHEEL_SPEED_RR_DELTA_KPH	WHEEL_SPEED_FL_DELTA_KPH	WHEEL_SPEED_FR_DELTA_KPH	YAWRATE	ODO	STEERINGANGLE	STEERINGANGLERATE	FIXTYPE[COMBUSTION	CSC	ELECTRIC	HYBRID]	SUPPORTLEVEL	DRIVERPOWER	ENGINEPOWER	BAT_CELSIUS"
    csv_columns = string.lower().split('\t')
    return

In [6]:
def nmea_import(filename):
    return

In [7]:
def gpx_import(filename):
    return

In [10]:
meta = {}
meta_dict = {}
track = widgets.Dropdown(
    options=['umc'],
    description='Track:',
    disabled=False,
)
track_config = widgets.Dropdown(
    options=['east','west','outer','full'],
    description='Configuration',
    disabled=False,
)
driver = widgets.Dropdown(
    options=['brandon','john','monte'],
    description='Driver',
    disabled=False,
)
date_pick = widgets.DatePicker(
    description='Date',
    disabled=False
)
session = widgets.Dropdown(
    options=['session 1','session 2','session 3','session 4','session 5', 'multiple sessions'],
    description='Session',
    disabled=False,
)
speed_units = widgets.Dropdown(
    options=['kph','mph'],
    description='Speed Units',
    disabled=False,
)
display(track)
display(track_config)
display(driver)
display(date_pick)
display(session)
display(speed_units)

Dropdown(description='Track:', options=('umc',), value='umc')

Dropdown(description='Configuration', options=('east', 'west', 'outer', 'full'), value='east')

Dropdown(description='Driver', options=('brandon', 'john', 'monte'), value='brandon')

DatePicker(value=None, description='Date')

Dropdown(description='Session', options=('session 1', 'session 2', 'session 3', 'session 4', 'session 5', 'mul…

Dropdown(description='Speed Units', options=('kph', 'mph'), value='kph')

In [29]:
meta_id = int(time.time())
meta_dict = {}
meta_dict['id'] = [meta_id]
meta_dict['track'] = [track.value]
meta_dict['track_config'] = [track_config.value]
meta_dict['track_and_config'] = ["%s-%s" % (track.value,track_config.value)]
meta_dict['driver'] = [driver.value]
meta_dict['date'] = [date_pick.value.strftime("%Y-%m-%d")]
meta_dict['session'] = [session.value]
meta_df = pd.DataFrame.from_dict(meta_dict)

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
#print(filename)
ext = splitext(filename)[1]
if ext == '.tsv':
    raw_data = tsv_import(filename, meta_id, speed_units.value,column_name_dict)
elif ext == '.vbo':
    print('vbo')
    vbo_import(filename, meta_id)
else:
    print('file type not compatible')
    
conn = sql.connect('race_data.db')
meta_df.to_sql('upload_meta', conn, if_exists='append')
raw_data.to_sql('raw_laps', conn, if_exists='append')


In [22]:
column_name_dict = {
     'lapindex':'lap_number',
     'date':'start_date',
     'time':'start_time',
     'time_lap':'lap_time',
     'latitude':'latitude',
     'longitude':'longitude',
     'speed_kph':'kph',
     'speed_mph':'mph',
     'height_m':'height_m',
     'height_ft':'height_ft',
     'heading_deg':'heading',
     'gpsdifferential[unknown/2d3d/dgps/invalid]':'gps_differential',
     'gpsfix[nofix/2d/3d/virtual/indoor]':'gps_fix',
     'gpsinterpolated[false/true]':'gps_interpolated',
     'satellites':'num_satellites',
     'hdop':'hdop',
     'accuracy_m':'accuracy_m',
     'distance_km':'distance_km',
     'distance_mile':'distance_mile',
     'accelerationsource[calculated/measured/undefined]':'accel_source',
     'lateralg':'gps_lat_acceleration',
     'linealg':'gps_lon_acceleration',
     'lean':'lean',
     'rpm':'rpm',
     'maf':'maf',
     'wheel_speed_kph':'wheel_speed_kph',
     'wheel_speed_mph':'wheel_speed_kmh',
     'throttle':'throttle_position',
     'gear':'gear',
     'fuel':'fuel',
     'coolant_celsius':'coolant_celsius',
     'oil_celsius':'oil_celsius',
     'iat_celsius':'iat_celsius',
     'map':'map',
     'oippressure':'oil_pressure',
     'brakepressure':'brake_pressure',
     'wheel_speed_rl_delta_kph':'wheel_speed_rl',
     'wheel_speed_rr_delta_kph':'wheel_speed_rr',
     'wheel_speed_fl_delta_kph':'wheel_speed_fl',
     'wheel_speed_fr_delta_kph':'wheel_speed_fr',
     'yawrate':'yaw_rate',
     'odo':'odo',
     'steeringangle':'steering_angle',
     'steeringanglerate':'steering_angle_rate',
     'fixtype[combustion,csc,electric,hybrid]':'fix_type',
     'supportlevel':'support_level',
     'driverpower':'driver_power',
     'enginepower':'engine_power',
     'bat_celsius':'bat_celsius',
     'start date': 'start_date', 
     'start time':'start_time', 
     'lap number':'lap_number', 
     'session time':'session_time', 
     'lap time':'lap_time',
     'longitudinal acceleration':'long_acceleration', 
     'gps lateral acceleration':'gps_lat_acceleration',
     'heading':'heading', 
     'gps distance':'gps_distance', 
     'gps longitudinal acceleration':'gps_long_acceleration',
     'lateral acceleration':'lat_acceleration', 
     'gps speed':'gps_speed',
     'number of satellites':'num_satellites', 
     'throttle position':'throttle_position'
    }